[GDCV06] GO/STOP! - Object Detection 시스템 만들기

<<목차>>

1) 자율주행 시스템 만들기

2) 자율주행 시스템 평가하기

3) 루브릭

4) 회고

1) 자율주행 시스템 만들기

In [ ]:
import os, copy
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

DATA_PATH = '/home/june/Github/GDCV/data/gd06'

: 

3) 루브릭

1. KITTI 데이터셋에 대한 분석이 체계적으로 진행되었다.

KITTI 데이터셋 구조와 내용을 파악하고 이를 토대로 필요한 데이터셋 가공을 정상 진행하였다.


2. RetinaNet 학습이 정상적으로 진행되어 object detection 결과의 시각화까지 진행되었다.

바운딩박스가 정확히 표시된 시각화된 이미지를 생성하였다.


3. 자율주행 Object Detection 테스트시스템 적용결과 만족스러운 정확도 성능을 달성하였다.

테스트 수행결과 90% 이상의 정확도를 보였다.

4) 회고

- 이번 프로젝트에서 **어려웠던 점,**

- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.

- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.

- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.

- **자기 다짐**